Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-21 20:16:08--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-21 20:16:09--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   115KB/s    in 1.8s    

2021-10-21 20:16:12 (115 KB/s) - ‘data.zip’ saved [217313/217313]



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


In [5]:
import os
import time
import random
import spacy
import dill
from tqdm import tqdm
import datetime
import numpy as np
import pandas
import logging
from argparse import ArgumentParser
from pdb import set_trace
import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

In [2]:
## Various utility functions
class args_class:
    dataset = "data/"
    model = "biLSTM"
    gpu = 0
    batch_size = 32
    epochs = 20
    hidden_dim = 64
    lr = 0.001
    results_dir = "2018EE10957_B_model"

In [3]:
def set_seed(seed=4):
	# fixing seeds to reproduce results (exact!)
	torch.manual_seed(seed)
	random.seed(seed)
	np.random.seed(seed)
    
def parse_args():
	# parser = ArgumentParser(description='NLI Baseline')
	# parser.add_argument('--dataset', '-d', type=str, default='mnli')
	# parser.add_argument('--model', '-m', type=str, default='bilstm')
	# parser.add_argument('--gpu', type=int, default=0)
	# parser.add_argument('--batch_size', type=int, default=128)
	# parser.add_argument('--embed_dim', type=int, default=300)
	# parser.add_argument('--epochs', type=int, default=20)
	# parser.add_argument('--lr', type=float, default=0.001)
	# parser.add_argument('--results_dir', type=str, default='results')
	args = args_class()
	return check_args(args)

"""checking arguments"""
def check_args(args):
	# --result_dir
	check_folder(os.path.join(args.dataset))
	check_folder(os.path.join(args.results_dir))

	# --epoch
	try:
			assert args.epochs >= 1
	except:
			print('number of epochs must be larger than or equal to one')

	# --batch_size
	try:
			assert args.batch_size >= 1
	except:
			print('batch size must be larger than or equal to one')
	return args

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
												filename = "{}_{}.log".format(args.model, phase),
												format = '%(asctime)s - %(message)s', 
												datefmt='%d-%b-%y %H:%M:%S')
	return logging.getLogger(phase)

In [4]:
def process_data(path, train=True):
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)

en = spacy.load('en_core_web_sm')
def tokeni(sen):
    t = en.tokenizer(sen)
    return [word.text for word in t]

class myDataset:
    def __init__(self, args):
        self.args = args
        
        process_data(args.dataset, train=True)
        process_data(args.dataset, train=False)

        TEXT = data.Field(
            sequential=True,
            lower=True,
            tokenize=tokeni, 
        )
        # get_tokenizer("basic_english"),
        fields = [
            ('word', TEXT),
            ('POS', data.Field(use_vocab=False, sequential=False)), 
            ('sen1', TEXT),
            ('word1', data.Field(use_vocab=False, sequential=False)),
            ('sen2', TEXT),
            ('word2', data.Field(use_vocab=False, sequential=False)),
            ('label', data.Field(use_vocab=False, sequential=False)),
        ]

        train_set, val_set = data.TabularDataset.splits(
            path = args.dataset,
            train = 'train/train.csv',
            validation = 'validation/validation.csv',
            format = 'csv',
            fields = fields,
            skip_header = False,
        )

        TEXT.build_vocab(train_set, vectors='glove.6B.300d')

        train_itr, val_itr = data.BucketIterator.splits(
            (train_set, val_set),
            #sort_key = lambda sample : len(sample.sen1),
            sort = False,
            batch_size = args.batch_size,
        )

        # my = TEXT.vocab.vectors
        # zero = torch.zeros(300)
        # zero_embs = ((my==zero).sum(axis=1) == 300)
        # zero_embs[0:2] = torch.tensor([False, False])
        # # zero_embs.sum()
        # TEXT.vocab.vectors[zero_embs] = torch.rand(zero_embs.sum(),300)
        # zero_embs = ((TEXT.vocab.vectors==zero).sum(axis=1) == 300)
        # # zero_embs[0:2] = torch.tensor([False, False])
        # print("zero embeddings in TEXT.vocab.vectors", zero_embs.sum())

        with open(os.path.join(self.args.results_dir, "Field_TEXT"), 'wb') as f:
            dill.dump(TEXT, f)
        print("TEXT Field saved in %s"%self.args.results_dir)
        
        self.vocab = TEXT.vocab
        self.train_iter = train_itr
        self.dev_iter = val_itr
        self.TEXT = TEXT
        self.train_set = train_set
        self.val_set = val_set

In [5]:
class myModel(nn.Module):
    def __init__(self, pretrained_embeddings, embed_dim=100, hidden_dim=100, device=torch.device('cpu')):
        super(myModel, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.device = device
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        # self.dropout = nn.Dropout(p = 0.5)
        self.bilstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=False, batch_first=True, num_layers=2)
        # self.fc = nn.Linear(hidden_dim, 2)
        # self.attn = nn.MultiheadAttention(embed_dim=300, num_heads=4, batch_first=True) #self.hidden_dim
        self.similarity = nn.CosineSimilarity(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        '''
            [torchtext.legacy.data.batch.Batch of size 32]
                [.word]:[torch.LongTensor of size 1x32]
                [.POS]:[torch.LongTensor of size 32]
                [.sen1]:[torch.LongTensor of size 17x32]
                [.word1]:[torch.LongTensor of size 32]
                [.sen2]:[torch.LongTensor of size 26x32]
                [.word2]:[torch.LongTensor of size 32]
                [.label]:[torch.LongTensor of size 32]
        '''
        input1 = input.sen1.permute(1,0).to(self.device)
        input2 = input.sen2.permute(1,0).to(self.device)

        # embed the tokens (indices in the vocab) 
        # out1 = self.dropout(self.embedding(input1))
        # out2 = self.dropout(self.embedding(input2))
        out1 = self.embedding(input1)
        out2 = self.embedding(input2)
        
        out1, (_, _) = self.bilstm(out1)
        out2, (_, _) = self.bilstm(out2)
        
        # mask1 = (input1 == 1)
        # mask2 = (input2 == 1)

        # attention box
        # (out1, _) = self.attn(out1, out1, out1, key_padding_mask=mask1)
        # (out2, _) = self.attn(out2, out2, out2, key_padding_mask=mask2)
        
        # word of interest
        woi1_idx = torch.repeat_interleave(torch.unsqueeze(torch.unsqueeze(input.word1, 1), 2), self.hidden_dim, dim=2).to(self.device)
        woi2_idx = torch.repeat_interleave(torch.unsqueeze(torch.unsqueeze(input.word2, 1), 2), self.hidden_dim, dim=2).to(self.device)
        
        # gather the word of interest for each sentence in the batch
        out1 = torch.gather(input=out1, dim=1, index=woi1_idx).squeeze()
        out2 = torch.gather(input=out2, dim=1, index=woi2_idx).squeeze()
        
        # linear layer
        #out1 = self.fc(out1)
        #out2 = self.fc(out2)
        
        # compute scores of similarity
        out = self.similarity(out1, out2)
        out = self.sigmoid(out)
        # try:
        #     assert out.shape[0] == self.bs
        # except:
        #     print("AssertionError", out.shape[0], self.bs) # all samples are touched in a epoch

        return out

In [6]:
set_seed()

class Train():
    def __init__(self):
        print("program execution start: {}".format(datetime.datetime.now()))
        self.args = parse_args()
        self.device = get_device(self.args.gpu)
        self.logger = get_logger(self.args, "train")
        self.logger.info("Arguments: {}".format(self.args))

        dataset_options = {
            'batch_size': self.args.batch_size, 
            'device': self.device
            }

        self.dataset = myDataset(self.args)
        self.vocab = self.dataset.vocab
        self.embed_dim = self.vocab.vectors.shape[1]

        self.model = myModel(self.vocab.vectors, self.embed_dim, self.args.hidden_dim, self.device)

        self.model = self.model.to(self.device)
        self.criterion = nn.BCELoss()
        self.opt = O.Adam(self.model.parameters(), lr = self.args.lr) #, weight_decay=0.005)
        self.best_val_acc = None
        # self.scheduler = StepLR(self.opt, step_size=5, gamma=0.5)

        print("resource preparation done: {}".format(datetime.datetime.now()))

    def train(self,epoch):
        self.model.train() #; self.dataset.train_iter.init_epoch()
        n_correct, n_total, n_loss = 0, 0, 0
        for batch_idx, batch in enumerate(self.dataset.train_iter):
            # if batch.batch_size != self.args.batch_size:
            #     print(batch.batch_size)
            self.opt.zero_grad()
            batch.label = batch.label.to(self.device)
            answer = self.model(batch)
            loss = self.criterion(answer, batch.label.float())
            answer = (answer>=0.5).long()

            n_correct += (answer == batch.label).sum().item()
            n_total += batch.batch_size
            n_loss += loss.item()

            loss.backward()
            # nn.utils.clip_grad_norm_(self.model.parameters(), 1, error_if_nonfinite=True)
            self.opt.step()
        # if epoch == 20:
            # set_trace()
        train_loss = n_loss/n_total
        train_acc = 100. * n_correct/n_total
        return train_loss, train_acc

    def validate(self,epoch):
        self.model.eval() #; self.dataset.dev_iter.init_epoch()
        n_correct, n_total, n_loss = 0, 0, 0
        with torch.no_grad():
            for batch_idx, batch in enumerate(self.dataset.dev_iter):
                batch.label = batch.label.to(self.device)
                answer = self.model(batch)
                loss = self.criterion(answer, batch.label.float())
                answer = (answer>=0.5).long()

                n_correct += (answer == batch.label).sum().item()
                n_total += batch.batch_size
                n_loss += loss.item()

            val_loss = n_loss/n_total
            val_acc = 100. * n_correct/n_total
            return val_loss, val_acc

    def result_checkpoint(self, epoch, train_loss, val_loss, train_acc, val_acc, took):
        if self.best_val_acc is None or val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            torch.save({
                'accuracy': self.best_val_acc,
                'model_dict': self.model.state_dict(),
            }, '{}/best-{}-params.pt'.format(self.args.results_dir, self.args.model))
        self.logger.info('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'
                .format(epoch, train_loss, train_acc, val_loss, val_acc, took))

    def execute(self):
        print(" [*] Training starts!")
        print('-' * 99)
        pbar = tqdm(range(1, self.args.epochs+1))
        for epoch in pbar:
            start = time.time()

            train_loss, train_acc = self.train(epoch)
            val_loss, val_acc = self.validate(epoch)
            # self.scheduler.step()

            took = time.time()-start
            self.result_checkpoint(epoch, train_loss, val_loss, train_acc, val_acc, took)

            pbar.set_description('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'.format(
                epoch, train_loss, train_acc, val_loss, val_acc, took))
        self.finish()

    def finish(self):
        self.logger.info("[*] Training finished!\n\n")
        print('-' * 99)
        print(" [*] Training finished!")
        print("best validation accuracy:", self.best_val_acc)

In [7]:
## Start training
task = Train()
task.execute()

program execution start: 2021-10-23 03:02:41.674205
TEXT Field saved in 2018EE10957_B_model
resource preparation done: 2021-10-23 03:02:46.979472
 [*] Training starts!
---------------------------------------------------------------------------------------------------


| Epoch  20 | train loss  0.01 | train acc 91.71 | val loss  0.02 | val acc 58.78 | time:  1.79s |: 100%|█| 20/20 

---------------------------------------------------------------------------------------------------
 [*] Training finished!
best validation accuracy: 63.47962382445141


## debugging

In [1]:
from transformers import AutoTokenizer

In [64]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [67]:
# print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.mask_token_id)

101 102 0 103


In [13]:
# tokenizer.encode(["hey my name is shubham", "they 'll play this game very hastily"], add_special_tokens=True)
print(tokenizer.convert_tokens_to_ids(["[PAD]", "[SEP]", "[CLS]", "[MASK]"]))
print(tokenizer.convert_ids_to_tokens([0,100,101, 102, 103, 104]))

[0, 102, 101, 103]
['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '[unused99]']


In [3]:
def process_data(path, train=True):
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False
    return dataset

In [17]:
df = process_data("./data/", train=False)

In [21]:
df.iloc[0]

word                                     board
POS                                          1
sen1                          Room and board .
word1                                        2
sen2     He nailed boards across the windows .
word2                                        2
label                                        0
Name: 0, dtype: object

In [79]:
out = tokenizer(["heyhey myhey","hi","hiello"], add_special_tokens=True, return_tensors='pt', padding='max_length', max_length=10) #, truncation=True)
out

{'input_ids': tensor([[  101,  4931, 14844,  2026, 14844,   102,     0,     0,     0,     0],
        [  101,  7632,   102,     0,     0,     0,     0,     0,     0,     0],
        [  101,  7632, 15350,   102,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}

In [70]:
# tokens = tokenizer.convert_ids_to_tokens(out['input_ids'][0])
# print(tokens)
# sen = tokenizer.convert_tokens_to_string(tokens)
# print(sen)
# print(" ".join(tokens))

In [80]:
from transformers import AutoModelForSequenceClassification

In [81]:
bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [83]:
bert = bert.to('cuda')

In [92]:
from torch.utils.data import Dataset, DataLoader
class WiC_dataset(Dataset):
    def __init__(self, data_dir, max_len=128, train=True, tokenizer='bert-base-uncased'):
        self.data_dir = data_dir
        self.max_len = max_len
        self.train = train

        # convert raw data into pandas data frame
        self.dataset_df = process_data(data_dir, train=train)

        # tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer, do_lower_case=True)

    def __getitem__(self, index):
        sample = self.dataset_df.iloc[index]

        tokenized = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', \
                                add_special_tokens=True, max_length=self.max_len, padding='max_length')
        
        x = tokenized['input_ids']
        y = torch.tensor(sample['label'])
        return x,y
    
    def __len__(self):
        return len(self.dataset_df)

In [93]:
trainset = WiC_dataset("data", max_len=128, train=True, tokenizer='bert-base-uncased')
trainloader = DataLoader(trainset, batch_size = 32, shuffle = True)

In [94]:
for x,y in trainloader:
    print(x.shape, y.shape)
    break

torch.Size([32, 1, 128]) torch.Size([32])


In [ ]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018EE10957_A_model.zip 2018EE10957_A_model

## Upload it to Google drive and ensure that the testing notebook uses the correct link

  adding: 2018EE10957_A_model/ (stored 0%)
  adding: 2018EE10957_A_model/best-biLSTM-params.pt (deflated 9%)
  adding: 2018EE10957_A_model/Field_TEXT (deflated 11%)


## testing

In [ ]:
def main():
    hidden_dim=64
    device = get_device(0)

    x_test = pandas.read_csv("test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    x_test = x_test.drop("position", axis=1)
    x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    x_test.insert(3, "word1", word1)
    x_test.insert(5, "word2", word2)
    x_test.to_csv("./test.csv", header=None, index=False)

    with open(os.path.join("2018EE10957_A_model", "Field_TEXT"), 'rb') as f:
        TEXT = dill.load(f)

    fields = [
        ('word', TEXT),
        ('POS', data.Field(use_vocab=False, sequential=False)), 
        ('sen1', TEXT),
        ('word1', data.Field(use_vocab=False, sequential=False)),
        ('sen2', TEXT),
        ('word2', data.Field(use_vocab=False, sequential=False)),
    ]

    test_set = data.TabularDataset(
        path = "./test.csv",
        format = 'csv',
        fields = fields,
        skip_header = False
    )

    test_itr = data.BucketIterator(
        test_set,
        #sort_key = lambda sample : len(sample.sen1),
        sort = False,
        batch_size = 32,
        repeat = False,
        shuffle = False,
    )

    model = myModel(pretrained_embeddings=TEXT.vocab.vectors, embed_dim=TEXT.vocab.vectors.shape[1], hidden_dim=hidden_dim, device=device)
    saved_model = torch.load("2018EE10957_A_model/best-biLSTM-params.pt", map_location=device)
    print(saved_model["accuracy"])
    model.load_state_dict(saved_model['model_dict'])
    model = model.to(device)

    model.eval()
    test_itr.init_epoch()
    predictions = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_itr):
            answer = model(batch)
            answer = list(np.array((answer>=0.5).long().cpu()))
            predictions += answer

    predictions = pandas.DataFrame(predictions)[0].apply(lambda label : "T" if label == 1 else "F")
    predictions.to_csv("output.txt", header=None, index=None)
    print("predictions written in output.txt")

In [ ]:
main()

60.18808777429467
predictions written in output.txt
